In [4]:
!pip install wiki_ru_wordnet

In [5]:
import pandas as pd
import re
import pymorphy2
import math
import pdb
import pickle
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()
morph = pymorphy2.MorphAnalyzer()

In [ ]:
df = pd.read_excel('Датасет. Задача 1.xlsx')

In [1]:
category_columns = ['Номинация / Задача', 'категория', 'Иновация']
label = 'Название'
word_clouds = {}

In [19]:
def get_hypernyms(words):
    new_cloud = []
    for word in words:
        lemmas = []
        synsets = wikiwordnet.get_synsets(word)
        for synset in synsets:
            for w in synset.get_words():
                lemmas.append(w.lemma())
                for hypernym in wikiwordnet.get_hypernyms(synset):
                    for h in hypernym.get_words():
                        lemmas.append(h.lemma())
                for hyponym in wikiwordnet.get_hyponyms(synset):
                    for h in hyponym.get_words():
                        lemmas.append(h.lemma())
        lemmas = list(set(lemmas))
        new_cloud.extend(lemmas)
    return new_cloud

In [60]:
cloud = df['words'][0]
print(cloud)
get_hypernyms(cloud)

['воздух', 'экологический', 'атмосферный', 'мониторинг']
воздух
экологический
атмосферный
мониторинг


['воздух',
 'смесь',
 'атмосфера',
 'атмосферный',
 'атмосферический',
 'слежка',
 'видеоконтроль',
 'мониторинг',
 'наблюдение',
 'надзор',
 'видеонаблюдение',
 'видеомониторинг']

In [10]:
def text_to_words(text):
    text = re.sub("[,|-|(|)|.|;|:|1|2|3|4|5|6|7|8|9|0]","", text)
    text = text.replace('-', ' ')
    text = text.lower()
    words = text.split(' ')
    words = map(morph.normal_forms, words)
    words = list(map(lambda x: x[0], words))
    words = list(set(words))
    return words
df['words'] = df['Название'].apply(text_to_words)

In [11]:
df_woc = pd.DataFrame({'category': pd.DataFrame(df['Номинация / Задача'].value_counts()).index})
df_woc

,category
0,Open Call
1,Меняющие реальность
2,Проект будущего
3,Лидеры инноваций
4,Рекомендательный сервис для оценки эффективнос...
5,Автоматизированный алгоритм обезличивания данных
6,Интерактивная карта для размещения спортивной ...
7,Персонализированная рекомендательная система п...
8,Рекомендательная система для пользователей мос...
9,Решение для распознавания и разметки пешеходны...


In [12]:
# create clouds for each category
def create_clouds(category):
    df_c = df[df['Номинация / Задача'] == category]
    claud = []
    df_c['words'].apply(lambda x: claud.extend(x))
    return claud

df_woc['clouds'] = df_woc['category'].apply(create_clouds)

In [15]:
def clear_clouds(row):
    category, cloud = row['category'], row['clouds']
    new_cloud = cloud.copy()
    for w in cloud:
        other_clouds = list(df_woc[df_woc['category'] != category]['clouds'])
        for orher_cloud in other_clouds:
            if w in orher_cloud:
                new_cloud.pop(new_cloud.index(w))
                break
    return new_cloud

df_woc['clear_clouds'] = df_woc.apply(clear_clouds, axis=1)

In [32]:
df_test = pd.read_excel('test_dataset.xlsx')
df_test.loc[0]

Название    Платформа для содействия внедрению новых техно...
Name: 0, dtype: object

In [20]:
df_woc['big_cloud'] = df_woc['clear_clouds'].apply(get_hypernyms)

In [21]:
df_woc

,category,clouds,clear_clouds,big_cloud
0,Open Call,"[позволять, крупный, избавить, персонал, засел...","[позволять, крупный, избавить, персонал, засел...","[сметь, позволять, разрешать, дозволять, дерза..."
1,Меняющие реальность,"[материал, обработка, экологически, и, энергоэ...","[экологически, финишный, текстильный, солнечны...","[экологически, научно, переносной, портативный..."
2,Проект будущего,"[, полимер, материал, тазобедренный, сустав, к...","[полимер, тазобедренный, сустав, разработка, э...","[пластмасса, полимер, протеин, фотополимер, по..."
3,Лидеры инноваций,"[мониторинг, воздух, атмосферный, экологически...","[воздух, атмосферный, экологический, управлени...","[смесь, атмосфера, воздух, атмосферический, ат..."
4,Рекомендательный сервис для оценки эффективнос...,"[предсказания\рекомендация, по, мы, конкретный...","[предсказания\рекомендация, конкретный, продви...","[конкретный, содействие, продвижение, промоушн..."
5,Автоматизированный алгоритм обезличивания данных,"[метод, предварительный, технический, мы, пост...","[метод, предварительный, технический, постанов...","[кольпоскопия, бегущая строка, метод, синтез, ..."
6,Интерактивная карта для размещения спортивной ...,"[, добавить, планировать, видео, связанный, не...","[добавить, видео, связанный, нет, спортивный, ...","[добавить, дорисовать, подпрячь, увеличить, до..."
7,Персонализированная рекомендательная система п...,"[сайт, отслеживание, как, возможность, совет, ...","[отслеживание, возможность, совет, смочь, друг...","[риск, возможность, превосходство, преимуществ..."
8,Рекомендательная система для пользователей мос...,"[кабинет, планировать, описание, библиотек\nра...","[кабинет, описание, библиотек\nразработать, ис...","[кабинет, правительство, комната, примат, заси..."
9,Решение для распознавания и разметки пешеходны...,"[как, по, камера, можно, back, разработчик, до...","[камера, back, дорога, дорожка, cv, видеонаблю...","[камера, богомолье, поход, автостоп, дорога, д..."


In [2]:
K_cloud = 1
K_clear_cloud = 10
K_top = 3


def find_categoris(text):
    scores = {} # catedory: score
    words = text_to_words(text)
    categoris = list(pd.DataFrame(df_woc['category'].value_counts()).index)
    for c in categoris:
        score = 0
        cloud = list(df_woc[df_woc['category'] == c]['clouds'])[0]
        clear_cloud = list(df_woc[df_woc['category'] == c]['clear_clouds'])[0]
        for w in words:
            if w in cloud:
                score += K_cloud / math.log(len(cloud))
            if w in clear_cloud:
                score += K_clear_cloud / math.log(len(clear_cloud))
        scores[c] = score
    sorted_scores = dict(sorted(scores.items(),key=lambda item:item[1])[::-1][0:K_top])
    return sorted_scores

df_test['scores'] = df_test['Название'].apply(find_categoris)

NameError: name 'df_test' is not defined

In [9]:
df_test

,Название,scores
0,Платформа для содействия внедрению новых техно...,"{'Проект будущего': 4.246968195105087, 'Open C..."
1,Производство иновационных материалов на основе...,"{'Меняющие реальность': 0.7927721098093363, 'С..."
2,Сайт для повышения цыфровой грамотности среи п...,"{'Лидеры инноваций': 8.716208489027562, 'Персо..."
3,Система распознаваиния преступления на основе ...,{'Персонализированная рекомендательная система...
4,Платформа по контролю озеленения,"{'Open Call': 0.3965559027119433, 'Лидеры инно..."


In [37]:
class CategoryPredictor:
    def __init__(self, label='Название', category_columns=['Номинация / Задача', 'категория', 'Иновация']):
        self.category_columns=category_columns
        self.curent_category = category_columns[0]
        self.label = label
        self.word_clouds = {}

    def read_df(self):
        self.df = pd.read_excel('Датасет. Задача 1.xlsx')

    def get_hypernyms(self, words):
        new_cloud = []
        for word in words: 
            lemmas = []
            synsets = wikiwordnet.get_synsets(word)
            for synset in synsets:
                for w in synset.get_words():
                    lemmas.append(w.lemma())
                    for hypernym in wikiwordnet.get_hypernyms(synset):
                        for h in hypernym.get_words():
                            lemmas.append(h.lemma())
                    for hyponym in wikiwordnet.get_hyponyms(synset):
                        for h in hyponym.get_words():
                            lemmas.append(h.lemma())
            lemmas = list(set(lemmas))
            new_cloud.extend(lemmas)
        return new_cloud

    def text_to_words(self, text):
        text = re.sub("[,|-|(|)|.|;|:|1|2|3|4|5|6|7|8|9|0]","", text)
        text = text.replace('-', ' ')
        text = text.lower()
        words = text.split(' ')
        words = map(morph.normal_forms, words)
        words = list(map(lambda x: x[0], words))
        words = list(set(words))
        return words
    
    # create clouds for each category
    def create_clouds(self, category):
        df_c = self.df[self.df[self.curent_category] == category]
        claud = []
        df_c['words'].apply(lambda x: claud.extend(x))
        return claud

    def clear_clouds(self, row):
        category, cloud = row['category'], row['clouds']
        new_cloud = cloud.copy()
        for w in cloud:
            other_clouds = list(self.woc[self.woc['category'] != category]['clouds'])
            for orher_cloud in other_clouds:
                if w in orher_cloud:
                    new_cloud.pop(new_cloud.index(w))
                    break
        return new_cloud

    def find_categoris(self, text, category=None, K_cloud=1, K_clear_cloud=10, K_big_cloud=1, K_top=3):
        if category != None:
            self.woc = self.word_clouds[category]
        scores = {} # catedory: score
        words = self.text_to_words(text)
        categoris = list(pd.DataFrame(self.woc['category'].value_counts()).index)
        for c in categoris:
            score = 0
            cloud = list(self.woc[self.woc['category'] == c]['clouds'])[0]
            clear_cloud = list(self.woc[self.woc['category'] == c]['clear_clouds'])[0]
            big_cloud = list(self.woc[self.woc['category'] == c]['big_clouds'])[0]
            for w in words:
                if w in cloud:
                    score += K_cloud / math.log(len(cloud))

                if w in clear_cloud:
                    score += K_clear_cloud / math.log(len(clear_cloud))
                
                if w in clear_cloud:
                    score += K_big_cloud / math.log(len(big_cloud))

            scores[c] = score
        sorted_scores = dict(sorted(scores.items(),key=lambda item:item[1])[::-1][0:K_top])
        return sorted_scores
    
    def fit(self, category_column):
        self.curent_category = category_column
        self.df['words'] = self.df[self.label].apply(self.text_to_words)
        woc = pd.DataFrame({'category': pd.DataFrame(self.df[category_column].value_counts()).index})
        self.woc = woc
        woc['clouds'] = woc['category'].apply(self.create_clouds)
        woc['clear_clouds'] = woc.apply(self.clear_clouds, axis=1)
        woc['big_clouds'] = woc['clear_clouds'].apply(self.get_hypernyms)
        self.word_clouds[category_column] = woc

    def predict(self, labels, category): # labels is dataframe
        self.woc = self.word_clouds[category]
        labels['predictions'] = labels[self.label].apply(self.find_categoris)
        return labels

    def fit_all(self):
        for category_column in self.category_columns:
            self.fit(category_column)


In [39]:
Predictor = CategoryPredictor(label='Название',
                              category_columns=['Номинация / Задача', 'категория', 'Иновация'])
Predictor.read_df()
Predictor.fit_all()
#Predictor.find_categoris('Платформа для содействия внедрению новых технологий', 'Номинация / Задача')
Predictor.predict(labels=df_test, category='Номинация / Задача')


,Название,predictions
0,Платформа для содействия внедрению новых техно...,"{'Проект будущего': 4.188132992580524, 'Open C..."
1,Производство иновационных материалов на основе...,"{'Меняющие реальность': 0.7927721098093363, 'С..."
2,Сайт для повышения цыфровой грамотности среи п...,"{'Лидеры инноваций': 9.293286505383147, 'Персо..."
3,Система распознаваиния преступления на основе ...,{'Персонализированная рекомендательная система...
4,Платформа по контролю озеленения,"{'Open Call': 0.39216713384653834, 'Лидеры инн..."
5,Интернет магазин пр продаже игрушек,"{'Open Call': 7.601879855717536, 'Чат-бот помо..."


In [30]:
df

NameError: name 'df' is not defined

In [29]:
Predictor.word_clouds['Номинация / Задача']

,category,clouds,clear_clouds,big_clouds
0,Open Call,"[киоск, и, избавить, участие, счёт, без, снизи...","[киоск, избавить, участие, счёт, без, снизить,...","[будка, ларёк, киоск, союзпечать, излечить, ис..."
1,Меняющие реальность,"[экологически, технология, материал, обработка...","[экологически, финишный, текстильный, батарея,...","[экологически, научно, батарея, комплекс, ради..."
2,Проект будущего,"[, эндопротезирование, композиционный, разрабо...","[эндопротезирование, разработка, биосовместимы...","[разработка, полиэфир, пластическая масса, бел..."
3,Лидеры инноваций,"[воздух, атмосферный, мониторинг, экологически...","[воздух, атмосферный, экологический, транспорт...","[смесь, воздух, атмосфера, атмосферный, атмосф..."
4,Рекомендательный сервис для оценки эффективнос...,"[продвижение, предсказания\рекомендация, банк,...","[продвижение, предсказания\рекомендация, банк,...","[продвижение, содействие, промоушн, организаци..."
5,Автоматизированный алгоритм обезличивания данных,"[решение, и, задание, сформулировать, метод, п...","[задание, сформулировать, метод, прогноз, нача...","[задание, лапароскопия, циклограмма, рекурсия,..."
6,Интерактивная карта для размещения спортивной ...,"[, оставить, планировать, и, нет, ios, каждый,...","[оставить, нет, ios, mac, где, добавить, вид, ...","[запамятовать, оставить, забыть, не-а, нет, гд..."
7,Персонализированная рекомендательная система п...,"[спонсор, фильтр, предприниматель, результат, ...","[спонсор, фильтр, предприниматель, действие, п...","[спонсор, папик, покровитель, патрон, фильтр, ..."
8,Рекомендательная система для пользователей мос...,"[планировать, предпочтение, основа, использова...","[состояние, описание, скоринг, кабинет, отобра...","[благополучие, прерывание, благоденствие, бесп..."
9,Решение для распознавания и разметки пешеходны...,"[если, изображение, обработка, дорожка, и, ком...","[изображение, дорожка, компетенция, решать, fr...","[фотоснимок, фотоизображение, фотка, снимок, и..."


In [10]:
df_test.to_excel('predicted_categoris.xlsx')